In [49]:
def extract_restaurant_from_yelp(zipcode):
    import time
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    data=pd.DataFrame(columns = ["restaurant_name","category","rating","neighborhood","address","zipcode"])
    url="https://www.yelp.com/search?find_desc=Restaurants&find_loc=%s"%(zipcode)
    #print(url)
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    page_count = 0
    listings_count = 1
    while listings_count<=int(1357):
        #add the time delay to prevent the anti-web-scraping 
        time.sleep(5)
        url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=%s&start=%s"%(zipcode,page_count*10)
        response = requests.get(url)
        #print(response.status_code)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        restaurant_each_page = page_data_soup.find_all('div',class_="search-result natural-search-result")
        for j in range(len(restaurant_each_page)):
            listings_count+=1
            single_result = page_data_soup.find_all('div',class_="search-result natural-search-result")[j]
            #find name
            try:
                name = single_result.find('h3',class_="search-result-title").text.strip()
            except:
                name = 'Nan'
            #find category
            try:
                category = single_result.find('span',class_="category-str-list").text.strip()
            except:
                category = 'Nan'
            #find rating
            try:
                rating = single_result.find('img',class_=src="https://s3-media1.fl.yelpcdn.com/assets/srv0/yelp_design_web/8a6fc2d74183/assets/img/stars/stars.png").get("alt").split()[0]
            except:
                rating = 'Nan'
            #find neighborhood
            try:
                neighborhood = single_result.find('span',class_="neighborhood-str-list").text.strip()
            except:
                neighborhood = 'Nan'
            #find address and zip_code
            try:
                address = single_result.find('address').text.strip()
                zip_code = address[-1]
            except:
                address = 'Nan'
                zip_code = 'Nan'
            data.loc[data.shape[0]] = [name,category,rating,neighborhood,address,zip_code]
            print("data:",name,category,rating,neighborhood,address,zip_code)            
        page_count+=1
        print(page_count)
    return data